In [25]:
#Sytem
import sys,os
import numpy as np
import pandas as pd
import warnings
import time
import random
import IPython
warnings.filterwarnings('ignore')

#Visualidation
import sweetviz as sv
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

#EDA
import sweetviz as sv
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder



#model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from skopt import BayesSearchCV
from xgboost import XGBClassifier




print('-'*40)

print("Python version: {}". format(sys.version))

print("pandas version: {}". format(pd.__version__))

print("matplotlib version: {}". format(matplotlib.__version__))

print("NumPy version: {}". format(np.__version__))


print("scikit-learn version: {}". format(sklearn.__version__))
print('-'*40)


----------------------------------------
Python version: 3.9.10 (main, Feb 22 2022, 13:54:07) 
[GCC 11.2.0]
pandas version: 1.3.5
matplotlib version: 3.5.1
NumPy version: 1.21.5
scikit-learn version: 1.1.1
----------------------------------------


In [26]:
Target = "Perished"
PATH_LOG = "./log.csv"
PATH_df = "./df"
ID = 10
SEED = 42

In [27]:
df_train = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")
base = pd.read_csv("./sub/10_submission_RandomForestClassifier.csv")
hard = pd.read_csv("./sub/10_submission_Emsenble.csv")

In [28]:
len(hard)

418

In [29]:
AddTrain = base.index[(base.Perished == hard.Perished)].tolist()
ReEval = base.index[(base.Perished != hard.Perished)].tolist()
newdata = df_test.loc[AddTrain,:]
newdata.loc[AddTrain,"Perished"] = base.loc[AddTrain,"Perished"]
newtest = df_test.loc[ReEval,:]
k

In [30]:
len(newtest)

30

In [31]:
dataset = pd.concat([df_train,newdata],sort=False)
data = pd.concat([dataset,newtest],sort=False)

In [32]:
label = LabelEncoder()
ohe = OneHotEncoder()

In [33]:
data['Title'] = data['Name'].str.split(
        ", ", expand=True)[1].str.split(".", expand=True)[0]

title_names = (data.Title.value_counts() < 10)
data["Title"] = data["Title"].apply(lambda x: "other" if title_names.loc[x] == True else x)

In [34]:
for title in data.Title.unique():
    mean = data.loc[(data.Title.values == title),"Age"].mean()
    data.loc[(data.Title.values == title) & (data.Age.isnull()),"Age"] = mean

In [35]:
faremean = data.Fare.mean()
data["Fare"].fillna(faremean,inplace=True)
data['FareBin'] = pd.qcut(data['Fare'], 10)
data['Fare_Code'] = label.fit_transform(data['FareBin'])


In [36]:
N_Age_Code = 3
data.replace({'Sex':{'male':0,'female':1}},inplace=True)
data['Age_Code'] = label.fit_transform(pd.qcut(data['Age'], N_Age_Code))
data['Embarked_Code'] = label.fit_transform(data['Embarked'])

In [37]:
data["Embarked"].fillna('S',inplace=True)
embarked_ohe = pd.get_dummies(data.Embarked)
data = pd.concat([data, embarked_ohe], axis=1)

In [38]:
drop_columns = ["Cabin",'FareBin',"Title","Embarked","Ticket","Name","Embarked_Code","Fare_Code"]
Fdata = data.drop(columns=drop_columns)
train = Fdata[:len(dataset)]
test = Fdata[len(dataset):]

In [39]:
test.head()

,PassengerId,Perished,Pclass,Sex,Age,SibSp,Parch,Fare,Age_Code,C,Q,S
17,909,NaN,3,0,21.0,0,0,7.2250,0,1,0,0
18,910,NaN,3,1,27.0,1,0,7.9250,1,0,0,1
19,911,NaN,3,1,45.0,0,0,7.2250,2,1,0,0
20,912,NaN,1,0,55.0,1,0,59.4000,2,1,0,0
23,915,NaN,1,0,21.0,0,1,61.3792,0,1,0,0


In [40]:
X = train.iloc[:, 2:].values
y = train.iloc[:, 1].values
X_test = test.iloc[:, 2:].values
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
Modelname = "XGBClassifier"
XGB = XGBClassifier()
CV = 5
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
 
XGB = XGBClassifier()
param_grid = {
    'n_estimators':n_estimators,
    'objective':['binary:logistic'],
    'max_depth':max_depth,
    'early_stopping_rounds':[5,10,20,50]
}


In [42]:
Search = "GS"
xgb_cv= GridSearchCV(estimator=XGB,param_grid=param_grid,scoring='accuracy',cv=CV,return_train_score=True,n_jobs=-1)
xgb_cv.fit(X_train,y_train,eval_set=[(X_valid,y_valid)])

print('best score: {:0.3f}'.format(xgb_cv.score(X_valid, y_valid)))
print('best params: {}'.format(xgb_cv.best_params_))
print('best val score:  {:0.3f}'.format(xgb_cv.best_score_))

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.12.1-git20200711.33e2d80-dfsg1-0.6 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.12.1-git20200711.33e2d80-dfsg1-0.6 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  w

[0]	validation_0-logloss:0.55341
[1]	validation_0-logloss:0.47339
[2]	validation_0-logloss:0.42466
[3]	validation_0-logloss:0.40066
[4]	validation_0-logloss:0.38991
[5]	validation_0-logloss:0.38049
[6]	validation_0-logloss:0.37533
[7]	validation_0-logloss:0.37859
[8]	validation_0-logloss:0.38007
[9]	validation_0-logloss:0.38505
[10]	validation_0-logloss:0.38335
[0]	validation_0-logloss:0.54633
[1]	validation_0-logloss:0.46872
[2]	validation_0-logloss:0.42239
[3]	validation_0-logloss:0.39546
[4]	validation_0-logloss:0.37967
[5]	validation_0-logloss:0.37297
[6]	validation_0-logloss:0.37203
[7]	validation_0-logloss:0.36900
[8]	validation_0-logloss:0.37360
[9]	validation_0-logloss:0.37796
[10]	validation_0-logloss:0.37924
[11]	validation_0-logloss:0.38475
[0]	validation_0-logloss:0.55048
[0]	validation_0-logloss:0.54633
[1]	validation_0-logloss:0.47477
[1]	validation_0-logloss:0.46872
[2]	validation_0-logloss:0.42827
[2]	validation_0-logloss:0.42239
[3]	validation_0-logloss:0.40285
[3]	val

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.12.1-git20200711.33e2d80-dfsg1-0.6 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.12.1-git20200711.33e2d80-dfsg1-0.6 is an invalid version and will not be supported in a future release
  warnings.warn(


[1]	validation_0-logloss:0.47318
[10]	validation_0-logloss:0.36083
[2]	validation_0-logloss:0.41173
[2]	validation_0-logloss:0.42796
[3]	validation_0-logloss:0.38344
[0]	validation_0-logloss:0.55048
[3]	validation_0-logloss:0.40155[11]	validation_0-logloss:0.36244

[4]	validation_0-logloss:0.37341[1]	validation_0-logloss:0.47477

[4]	validation_0-logloss:0.38447
[2]	validation_0-logloss:0.42827
[5]	validation_0-logloss:0.36469
[5]	validation_0-logloss:0.38248
[3]	validation_0-logloss:0.40285
[6]	validation_0-logloss:0.36011
[6]	validation_0-logloss:0.38183
[0]	validation_0-logloss:0.54931
[4]	validation_0-logloss:0.38852
[7]	validation_0-logloss:0.35878
[1]	validation_0-logloss:0.47318
[7]	validation_0-logloss:0.38151
[5]	validation_0-logloss:0.38361
[8]	validation_0-logloss:0.38368
[2]	validation_0-logloss:0.42796
[6]	validation_0-logloss:0.37843
[8]	validation_0-logloss:0.35889
[9]	validation_0-logloss:0.38772
[7]	validation_0-logloss:0.37589
[9]	validation_0-logloss:0.36087
[3]	vali

In [20]:
Search = "BS"
xgb_cv = BayesSearchCV(estimator=XGB,search_spaces=param_grid,n_iter=100,cv=CV,verbose =2,n_jobs=-1)
xgb_cv.fit(X_train,y_train,eval_set=[(X_valid,y_valid)])

print('best score: {:0.3f}'.format(xgb_cv.score(X_valid, y_valid)))
print('best params: {}'.format(xgb_cv.best_params_))
print('best val score:  {:0.3f}'.format(xgb_cv.best_score_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[0]	validation_0-logloss:0.56019
[0]	validation_0-logloss:0.54687
[1]	validation_0-logloss:0.48538
[0]	validation_0-logloss:0.55321
[0]	validation_0-logloss:0.56214
[1]	validation_0-logloss:0.47788
[2]	validation_0-logloss:0.44728
[0]	validation_0-logloss:0.55618
[1]	validation_0-logloss:0.47884
[3]	validation_0-logloss:0.42286
[1]	validation_0-logloss:0.48856
[2]	validation_0-logloss:0.43156
[2]	validation_0-logloss:0.43830[1]	validation_0-logloss:0.48428

[4]	validation_0-logloss:0.41234
[2]	validation_0-logloss:0.44757
[2]	validation_0-logloss:0.44769
[3]	validation_0-logloss:0.41517
[5]	validation_0-logloss:0.40779
[3]	validation_0-logloss:0.41418
[3]	validation_0-logloss:0.41700
[3]	validation_0-logloss:0.42824
[4]	validation_0-logloss:0.40371
[6]	validation_0-logloss:0.41029
[4]	validation_0-logloss:0.40478
[4]	validation_0-logloss:0.40413
[4]	validation_0-logloss:0.42125
[5]	validation_0-logloss:0.39809
[7]	validation_0-

In [43]:
best_model = xgb_cv.best_estimator_
print('Train Score: {}'.format(round(best_model.score(X_train, y_train), 4)))
print(' Test Score: {}'.format(round(best_model.score(X_valid, y_valid), 4)))

Train Score: 0.9374
 Test Score: 0.8555


In [44]:
pred = best_model.predict(X_test)
pred

array([1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0])

In [45]:
for i in range(0,len(ReEval)):
    base.loc[ReEval[i],"Perished"] = pred[i]


In [46]:
PATH_sub =  ("./sub/"+str(ID)+"_submission_"+"Final"+".csv")
print(PATH_sub)
submission_ex = pd.read_csv("gender_submission.csv")
base.to_csv(PATH_sub, header=True, index=False)
print("Submission Format:{} ".format(submission_ex.shape))
print("My Sumbisison Format:{}".format(base.shape))
base.head(5)

./sub/10_submission_Final.csv
Submission Format:(418, 2) 
My Sumbisison Format:(418, 2)


,PassengerId,Perished
0,892,1.0
1,893,1.0
2,894,1.0
3,895,1.0
4,896,1.0
